In  playing with font shapes, I noticed that what I was generated looked like the drawings of Eric Carle. I decided to make a psuedo-Carle scene out of manipulated fonts.

I began by using the flat, numpy, and bezmerizing imports:

In [69]:
from flat import document, rgb, rgba, font, strike, shape
from flat.command import moveto, quadto, curveto, lineto, closepath
from bezmerizing import Polyline, Path
import numpy as np
from copy import copy

In [70]:
from numpy.random import uniform, normal, choice

In [71]:
from IPython.display import SVG, display
def show(page):
    display(SVG(page.svg()))

In [72]:
def glyphcommands(f, ch):
    return Path([copy(cmd) for cmd in f.glyph(f.charmap[ord(ch)])])
def advancefor(f, ch):
    return f.advances[f.charmap[ord(ch)]]

I used the Google font Monofett for this exercise, available here:
https://fonts.google.com/specimen/Monofett?category=Serif,Display&width=7&thickness=10&selection.family=Monofett&sidebar.open

In [73]:
f = font.open("./Monofett-Regular.ttf")

In [74]:
all_indices = list(f.charmap.values())

In [75]:
glyph_path_lengths = [(idx, len(f.glyph(idx))) for idx in all_indices]

In [80]:
from itertools import chain
def combine_path(f, s):
    text_paths = []
    cx = 0
    for ch in s:
        glyph_path = glyphcommands(f, ch).translate(cx, 0)
        text_paths.append(glyph_path)
        cx += advancefor(f, ch)
    combined = Path(list(chain(*text_paths)))
    return combined

I decided to make blades of grass through "censored" rectangular shapes overlayed on words from a poem, and green color ranges based on the length of the words in that poem.

I used the following poem that I found on Reddit a while ago, written by an anonymous user:

I really hate this damn machine
I wish that I could sell it
It doesn't do what I want it to do
But only what I tell it.


In [90]:
width = 600 # 5in
height = 500 # 4in
border = 50 # 1/8in
lineheight = 22 # size of each line
rectsize = 60 # size of each rectangle
page = document(width, height, 'pt').addpage()
yoff = border + 425  
xoff = border - 100
pen = shape().nostroke().fill(rgba(107, 82, 57, 100))
factor = 150 / f.density
cx = 0
cy = 0

for line in open("machine.txt"):
    line = line.strip() # remove trailing whitespace
    for ch in line:
        path_len = len(glyphcommands(f, ch).commands)
        val = 105 - (path_len)
        advance = advancefor(f, ch) / f.density * lineheight*2
        brush = shape().nostroke().fill(rgb(val, 100 + val, val))
        page.place(brush.rectangle(xoff, yoff, advance, rectsize))
        xoff += int(advance)+5
    xoff = border
    yoff += lineheight*2.5
    
show(page)

I then created "dirt" by taking the phrase "Go Outside" and turning the path of the font and reinterpreting as circles and curves instead of lines, with varying opacity

In [91]:
from math import sqrt
d = document(600, 500, 'pt')
page = d.addpage()
pen = shape().nostroke().fill(rgba(107, 82, 57, 64))
factor = 102 / f.density
cx = 0
cy = 0
glyph_path = combine_path(f, "GO OUTSIDE").scale(factor).translate(50, 650)
for cmd in glyph_path:
    if type(cmd) in (lineto, curveto, quadto):
        radius = sqrt(pow(cx - cmd.x, 2) + pow(cy - cmd.y, 2)) * 2
        page.place(pen.circle((cx + cmd.x) / 2 +50, (cy + cmd.y) / 2 +50, radius*4))
    if type(cmd) != type(closepath):
        cx = cmd.x
        cy = cmd.y
show(page)

In [92]:
width = 600 # 5in
height = 500 # 4in
border = 50 # 1/8in
lineheight = 22 # size of each line
rectsize = 60 # size of each rectangle
page = document(width, height, 'pt').addpage()
yoff = border + 425  
xoff = border - 100
pen = shape().nostroke().fill(rgba(107, 82, 57, 100))
factor = 150 / f.density
cx = 0
cy = 0

figure = shape().fill(rgba(135, 206, 235, 255)).stroke(rgba(0, 0, 0, 0)).width(0)
rect = figure.rectangle(0, 0, 800, 800)
page.place(rect)

glyph_path = combine_path(f, "GO OUTSIDE").scale(factor).translate(50, 650)
for cmd in glyph_path:
    if type(cmd) in (lineto, curveto, quadto):
        radius = sqrt(pow(cx - cmd.x, 2) + pow(cy - cmd.y, 2)) * 2
        page.place(pen.circle((cx + cmd.x) / 2, (cy + cmd.y) / 2+100, radius*3))
    if type(cmd) != type(closepath):
        cx = cmd.x
        cy = cmd.y

for line in open("machine.txt"):
    line = line.strip() # remove trailing whitespace
    for ch in line:
        path_len = len(glyphcommands(f, ch).commands)
        val = 105 - (path_len)
        advance = advancefor(f, ch) / f.density * lineheight*2
        brush = shape().nostroke().fill(rgb(val, 100 + val, val))
        page.place(brush.rectangle(xoff, yoff, advance, rectsize))
        xoff += int(advance)+5
    xoff = border
    yoff += lineheight*2.5


show(page)

After creating my font scene, I tried to add a few more elements to my landscape - I wanted to make bugs, and include the sun emoji

In [94]:
from math import sqrt
d = document(288, 144, 'pt')
page = d.addpage()
pen2 = shape().stroke(rgb(191, 49, 55)).fill(rgb(191, 49, 55))
factor2 = 40 / f.density
cx = 0
cy = 0
glyph_path = glyphcommands(f, "*").scale(factor2).translate(102, 144)
for cmd in glyph_path:
    if type(cmd) == lineto:
        radius = sqrt(pow(cx - cmd.x, 2)*10 + pow(cy - cmd.y, 2)) * 0.7
        page.place(pen2.circle((cx + cmd.x) / 2, (cy + cmd.y) / 2, radius))
    elif type(cmd) in (curveto, quadto):
        page.place(pen.line(cx, cy, cmd.x, cmd.y))
    if type(cmd) != type(closepath):
        cx = cmd.x
        cy = cmd.y
show(page)

In [95]:
width = 600 # 5in
height = 500 # 4in
border = 50 # 1/8in
lineheight = 22 # size of each line
rectsize = 60 # size of each rectangle
page = document(width, height, 'pt').addpage()
yoff = border + 425  
xoff = border - 100
pen = shape().nostroke().fill(rgba(107, 82, 57, 100))
pen2 = shape().stroke(rgb(191, 49, 55)).fill(rgb(191, 49, 55))
factor = 150 / f.density
factor2 = 30 / f.density
cx = 0
cy = 0

figure = shape().fill(rgba(135, 206, 235, 255)).stroke(rgba(0, 0, 0, 255)).width(0)
rect = figure.rectangle(0, 0, 800, 800)
page.place(rect)

glyph_path = combine_path(f, "GO OUTSIDE").scale(factor).translate(50, 650)
for cmd in glyph_path:
    if type(cmd) in (lineto, curveto, quadto):
        radius = sqrt(pow(cx - cmd.x, 2) + pow(cy - cmd.y, 2)) * 2
        page.place(pen.circle((cx + cmd.x) / 2, (cy + cmd.y) / 2+100, radius*3))
    if type(cmd) != type(closepath):
        cx = cmd.x
        cy = cmd.y

for line in open("machine.txt"):
    line = line.strip() # remove trailing whitespace
    for ch in line:
        path_len = len(glyphcommands(f, ch).commands)
        val = 105 - (path_len)
        advance = advancefor(f, ch) / f.density * lineheight*2
        brush = shape().nostroke().fill(rgb(val, 100 + val, val))
        page.place(brush.rectangle(xoff, yoff, advance, rectsize))
        xoff += int(advance)+5
    xoff = border
    yoff += lineheight*2.5

glyph_path = glyphcommands(f, "*").scale(factor2).translate(102, 144)
for cmd in glyph_path:
    if type(cmd) == lineto:
        radius = sqrt(pow(cx - cmd.x, 2)*10 + pow(cy - cmd.y, 2)) * 0.7
        page.place(pen2.circle((cx + cmd.x) / 2, (cy + cmd.y) / 2, radius))
    elif type(cmd) in (curveto, quadto):
        page.place(pen.line(cx, cy, cmd.x, cmd.y))
    if type(cmd) != type(closepath):
        cx = cmd.x
        cy = cmd.y

show(page)

I have two components that I want to work on moving forward - first, I'd like to make better "bugs" of varying size and position. Below was the start of my playing with this style.

In [96]:
from math import sqrt
d = document(600, 200, 'pt')
page = d.addpage()
pen = shape().nostroke().fill(rgba(102, 0, 0, 245))
factor = 36 / f.density
cx = 40
cy = 500
glyph_path = combine_path(f, "* * * * * * *").scale(factor).translate(factor + uniform(300), factor + uniform(300))
for cmd in glyph_path:
    if type(cmd) in (lineto, curveto, quadto):
        radius = sqrt(pow(cx - cmd.x, 2) + pow(cy - cmd.y, 2)) * 0.2
        page.place(pen.circle((cx + cmd.x) / 3, (cy + cmd.y) , radius))
    if type(cmd) != type(closepath):
        cx = advancefor(f, ch)/ 10000 
        cy = cmd.y + 10
show(page)

It also appears that I cannot add sun emojis, regardless of what font I use. This is confusing because sun emojis have unicode and glyph codes.

In [89]:
z_path_orig = glyphcommands(f, "☀️")

TypeError: ord() expected a character, but string of length 2 found